In [2]:
import pandas as pd
import calendar
import plotly.express as px
import hvplot.pandas
from sqlalchemy import create_engine
import seaborn as sns
import csv
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime

import os
import psycopg2
from dotenv import load_dotenv

In [6]:
"""
This section is created to prep the data to generate a parallel
categories chart with columns: temp, precip, production, year
"""
# extract tot produc from csv file

path_crop = Path ('../data/raw_data/tot_corn_prod_year.csv')
csv_data = pd.read_csv(path_crop)
df =pd.DataFrame(data=csv_data)
df.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2020,YEAR,NaN,STATE,NEBRASKA,31,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"1,838,210,000",NaN
1,SURVEY,2019,YEAR,NaN,STATE,NEBRASKA,31,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"1,785,420,000",NaN
2,SURVEY,2018,YEAR,NaN,STATE,NEBRASKA,31,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"1,785,600,000",NaN
3,SURVEY,2017,YEAR,NaN,STATE,NEBRASKA,31,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"1,683,300,000",NaN
4,SURVEY,2016,YEAR,NaN,STATE,NEBRASKA,31,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",TOTAL,NOT SPECIFIED,"1,699,900,000",NaN


In [12]:
new_df = df[['Year','Commodity','Value']]
new_df.head()

,Year,Commodity,Value
0,2020,CORN,"1,838,210,000"
1,2019,CORN,"1,785,420,000"
2,2018,CORN,"1,785,600,000"
3,2017,CORN,"1,683,300,000"
4,2016,CORN,"1,699,900,000"


In [18]:
# sort from highest to lowest and keep top 10 years

new_df.sort_values('Value', ascending=False, inplace=True)
new_df.head()

,Year,Commodity,Value
18,2002,CORN,"940,800,000"
0,2020,CORN,"1,838,210,000"
2,2018,CORN,"1,785,600,000"
1,2019,CORN,"1,785,420,000"
4,2016,CORN,"1,699,900,000"
